# The three levels of aggregation

In the previous section, the groupby object only finishes the first step of split-apply-combine. The complexity of implementing the next two steps may vary. Here's the three type from simple to complex. (actually not that complex)

1. If we intend to calculate mean, max, or size, one just invoke them on the groupby object. It's been optimized.
2. If the method exists for each dataframe chunk, like quantile, but does not fall in the first category, we can still just call it. Maybe slower tho. Internally, groupby slices up the series and calls quantile function for each piece, and stiches them together into the result objects. 
3. If we want a customized method, we got to pass in a function to the agg function.

Hence, there is technical difference between the first two situations but to the end user, the interface is identical.

In [1]:
import pandas as pd 
tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [2]:
case_1 = tips.groupby(['day']).mean()[['total_bill','tip']]
case_1

,total_bill,tip
day,,
Fri,17.151579,2.734737
Sat,20.441379,2.993103
Sun,21.410000,3.255132
Thur,17.682742,2.771452


In [3]:
case_2 = tips.groupby(['day']).quantile(0.9)[['total_bill','tip']]
case_2.columns.names =['']
case_2

,total_bill,tip
day,,
Fri,27.618,4.060
Sat,31.894,4.802
Sun,33.765,5.035
Thur,28.316,4.920


In [4]:
def find_range(arr):
    return arr.max() - arr.min()

In [5]:
case_3 = tips.groupby(['day']).agg(find_range)[['total_bill','tip']]
case_3.head()

,total_bill,tip
day,,
Fri,34.42,3.73
Sat,47.74,9.00
Sun,40.92,5.49
Thur,35.60,5.45


# More operations and more variables

This section settles several questions:
1. what if there is more than one operation we desire to impose for one variable
2. what if for different variable we want to define a different operations

In [6]:
import numpy as np 
case2_1 = tips.groupby(['day']).agg(['std', np.mean, find_range])[['total_bill', 'tip']]
case2_1

total_bill                             tip                     
            std       mean find_range       std      mean find_range
day                                                                 
Fri    8.302660  17.151579      34.42  1.019577  2.734737       3.73
Sat    9.480419  20.441379      47.74  1.631014  2.993103       9.00
Sun    8.832122  21.410000      40.92  1.234880  3.255132       5.49
Thur   7.886170  17.682742      35.60  1.240223  2.771452       5.45

In [7]:
case2_1_names = tips.groupby(['day']).agg([('standard dev', np.std), ('mean',np.mean), ('range',find_range)])[['total_bill', 'tip']]
case2_1_names

total_bill                            tip                
     standard dev       mean  range standard dev      mean range
day                                                             
Fri      8.302660  17.151579  34.42     1.019577  2.734737  3.73
Sat      9.480419  20.441379  47.74     1.631014  2.993103  9.00
Sun      8.832122  21.410000  40.92     1.234880  3.255132  5.49
Thur     7.886170  17.682742  35.60     1.240223  2.771452  5.45

To recap, we simply need to pass a list of operations into the agg method. It allows a mixture of built-in functions, which you can call by giving a string ('mean', 'std' for instance), numpy aggregation functions and even customized funtions. 

Moreover, one can change the name of the output column by passing a tuple. The first arg of the tuple is name the second is the operation (function). The only caveat is the function used here has to be aggregation function, which produces scalar values from arrays. 

In [8]:
case_2_2 = tips.groupby(['day', 'smoker']).agg({'total_bill': [('max_value',np.max)], 'tip': [('range',find_range),('mean', 'mean')]})

In [9]:
case_2_2

total_bill   tip          
             max_value range      mean
day  smoker                           
Fri  No          22.75  2.00  2.812500
     Yes         40.17  3.73  2.714000
Sat  No          48.33  8.00  3.102889
     Yes         50.81  9.00  2.875476
Sun  No          48.17  4.99  3.167895
     Yes         45.35  5.00  3.516842
Thur No          41.19  5.45  2.673778
     Yes         43.11  3.00  3.030000

The answer is pass in a dictionary. The keys are names of columns to be aggregated, and the values are operations. If there is more than one operation, simply pass a list; if there is a need to customize the name of column, pass a tuple instead.

In [10]:
case_2_2.loc['Fri','No']

total_bill  max_value    22.7500
tip         range         2.0000
            mean          2.8125
Name: (Fri, No), dtype: float64